# Methods

## Introduction: Discrete vs. Continuous

It is natural to think of water flowing in a stream as a continuous quantity.  Water level in a stream rises and falls tracing a smooth line in time as opposed to stepping up and down abruptly at fixed time intervals.  When recording streamflow observations quantitatively, the continuous values representing streamflow at a moment in time are converted to a discrete form and stored on a computer in 32 or 64-bit floating point format.  These two formats can represent approximately $4.3\times 10^9$ (appriximately 7 decimal precision) and $1.8 \times 10^{19}$ distinct states, or 7 and 16 decimal precision respectively.  Streamflow observations don't need that much precision because measurement (rating curve) uncertainty is a limiting factor.

Streamflow measurement uncertainty is multiplicative, meaning the uncertainty varies *in proportion to the magnitude*.  The Water Survey of Canada published the [HYDAT](https://www.canada.ca/en/environment-climate-change/services/water-overview/quantity/monitoring/survey/data-products-services/national-archive-hydat.html) database of estimated daily (and hourly in some cases) streamflow at 1000+ stations in Canada.  Mean daily streamflow series from the HYDAT dataset uses 3 decimal precision.  Given an example range of 0.1 to 100 $m^3/s$, this precision suggests $(100-0.1) / 0.001 = 99900$ unique states, or roughly 17 bits ($2^{17} = 131,072$).

The figure below illustrates how increasing the number of states representing the observed series converges to the continuous function $y(t) = 5 + \sin(t) + 0.5\sin(3t)$. In the example below, even 5 bits gives a close representation of the continuous function, though this reflects in large part to the range of inputs and the nature of the function.  Click on the legend labels to toggle series and see the effect more clearly. 

In [8]:
import os
import numpy as np
import pandas as pd
import xyzservices.providers as xyz
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Sunset10, Vibrant7
output_notebook()

def discrete_series(wl, bits):
    min_w, max_w = np.min(wl)-1e-9, np.max(wl)+1e-9
    # edges_log = np.linspace(np.log10(min_w), np.log10(max_w), 2**bits)
    edges = np.linspace(min_w, max_w, 2**bits)
    
    # edges = np.array([10**e for e in edges_log])    
    midpoints = (edges[1:] + edges[:-1]) / 2

    # digits = np.digitize(wl, edges)
    digits = np.digitize(wl, edges) - 1
    digits = np.clip(digits, 0, len(midpoints) - 1)
    
    return midpoints[digits]
    
# Generate example data
time = np.linspace(0, 10, 500)
wl_0 = 5 + np.sin(time) + 0.5 * np.sin(3 * time)
p = figure(title="Discrete to continuous streamflow", width=800, height=300)

n = 0
for b in range(2,9):
    wl = discrete_series(wl_0, b)
    p.line(time, wl, color=Vibrant7[n], line_width=3,
           legend_label=str(b)+' bits')
    n += 1
p.line(time, wl_0, legend_label='continuous', line_dash='dashed', color='red',
      line_width=3)    
p.yaxis.axis_label = 'Water level'
p.add_layout(p.legend[0], 'right')
p.xaxis.axis_label = 'Time'
p.legend.click_policy='mute'
show(p)

Loading BokehJS ...

The introdution above describes two key assumptions in the methods of this study: the uncertainty in the observations themselves, and the detail or resolution in which we represent (uncertain) observations.  

### Streams as Systems with Discrete States

Streamflow can be viewed as a system observed in various states defined by $X$, where states are defined by monotonically increasing intervals over a range of flow.  The number and width of intervals determine how precisely the system states can represent streamflow, and how much information is lost when converting continuous streamflow observations to discrete values.  The effect of quantization is a key assumption tested in this study.

Signal quantization is the process of mapping a continuous range of signal values to a finite set of discrete levels. This is often done in digital signal processing to represent a signal with a limited number of bits. However, quantization introduces an error or "noise" because the original signal's fine details are lost when it is forced into discrete levels represented by a single value for each level. The quantization error represents the difference between the actual signal and the quantized version, introducing uncertainty into the representation of the signal.  


### Streamflow as a Noisy Signal

In hydrology, streamflow is derived from water level (stage) measurements using a stage-discharge relationship, often modeled by a power law or interpolation. This process, like quantization, introduces uncertainty into the estimation of streamflow. Quantization noise is in a limited sense analogous to rating curve uncertainty because in both cases error is introduced when mapping continuous observations to discrete values through an imperfect model.  {cite}`hamilton2012quantifying` describes many other sources of uncertainty in defining the stage-discharge relationship.

In this study we compute a number of information measures based on discrete distributions of streamflow timeseries, and test the effect of quantization on the predictability of each measure from catchment attributes.  If too many quantization levels are used, false precision is introduced and the quantization could be overfitting to noise.  In contrast, too few levels results in a loss of information by oversimplifying the data and underestimating variability in the signal.  By testing a wide range of quantization schemes, a balance is sought in quantization to reflect uncertainty in the noisy streamflow signal.

## Entropy and Distance Measures 

The class of functions that describe the difference between two probability distributions $P$ and $Q$ is defined in general as:

$$D_f(P||Q) = \int \mathcal{f} \left( \frac{dP}{dQ} \right) dQ$$




## Predicting Hydrological Signatures from Catchment Attributes

{cite}`mcmillan2021review` presents a comprehensive review of approaches to hydrological signature prediction from catchment attributes, covering a large number of signatures and their links to hydrological processes.  Several signatures relate to specific exceedance percentiles, which may be seen as first order characteristics since they represent positions in the FDC, and others may be seen as second order since they describe slopes (for example the slope of the FDC between the log-transformed 33rd and 66th streamflow percentiles).  The mean is a summary statistic in the sense that it encapsulates all observations.  F-divergence measures are more closely aligned with these latter signatures since they characterize the full distribution, however the test setup is different in the case of testing predictability of divergence metrics due to the target variable being a function of two probability distributions representing two locations.  

The capacity of catchment attributes to predict hydrological signatures has been shown to be linked to spatial smoothness {cite}`addor2018ranking`.  Two levels of model complexity are tested in this study.  The first is a binary classification model which tests the model's ability to predict whether a regional donor/proxy catchment provides a more informative model (smaller f-divergence) than the maximum uncertainty prior (uniform model Q).  The second is a nonlinear regression model that outputs predictions of the target variables.  A summary of tests is provided in {numref}`model-summary-table`.


```{list-table} Summary of Model Scenarios
:header-rows: 1
:name: model-summary-table

* - Number
  - Target Variable
  - Description
  - Variations
* - 1
  - Mean Daily Discharge (MD)
  - Predict the mean daily discharge (mm/day) using catchment attributes.  
  - Vary the ordering of catchment attribute groups to test influence of groups of related attributes, test randomized attributes.
* - 2
  - Shannon Entropy (H)
  - Predict H computed from unit area runoff distributions at single locations from catchment attributes.
  - Vary the encoding dictionary size between 4, 6, 8, 9, 10, 11, and 12 bits.
* - 3
  - KL Divergence
  - Predict the Kullback-Leibler (KL) divergence of pseudo-simulated (Q) from observed (P) unit area runoff distributions between pairs of locations from catchment attributes.
  - Vary the encoding dictionary size between 4 and 12 bits, vary the prior applied to Q, test concurrent and nonconcurrent periods of record, test the effect of probabilistic treatment of observation error (10% uniform).
* - 4
  - Earth Mover's Distance (EMD)
  - Predict the EMD between observed (Q) and pseudo-simulated (P) unit area runoff distributions between pairs of locations from catchment attributes.
  - Vary the encoding dictionary size between 4 and 12 bits, test concurrent and nonconcurrent periods of record, test the effect of probabilistic treatment of observation error (10% uniform).
* - 5
  - Total Variation Distance (TVD)
  - Predict the total variation distance between observed (Q) and pseudo-simulated (P) unit area runoff distributions between pairs of locations from catchment attributes.
  - Vary the encoding dictionary size between 4 and 12 bits, test concurrent and nonconcurrent periods of record, test the effect of probabilistic treatment of observation error (10% uniform).
```





### Mean Daily Discharge (MDD) & Mean (Unit Area) Runoff (MUR)
{cite}`addor2018ranking` predicts mean daily discharge from catchment attributes for approximately 600 catchments over the continental US from the CAMELS {cite}`arsenault2020comprehensive` dataset.  A similar approach is taken to replicate those results here:  

$$\text{MDD}(X) = \frac{1}{n} \sum_{i=1}^n X_i \quad mm/day$$

The conversion from $\frac{1 m^3}{s}$ to $86.4 \frac{mm}{day}$ is $\frac{1 m^3}{s} \cdot \frac{3600s}{1h} \cdot \frac{24h}{1day} \cdot \frac{1000mm}{1m} \cdot \frac{1 \text{km}^2}{(1000m)^2} \cdot \frac{1}{A_{catchment} \text{km}^2}$.

The MUR units are expressed in $\frac{L}{s \cdot \text{km}^2}$ converted from $\frac{m^3}{s}$ by multiplying by $\frac{1000 L}{m^3}$ and dividing by the drainage area ($A_{catchment}$) in $\text{km}^2$


### Shannon Entropy

{cite}`shannon1948mathematical` describes entropy as a measure of the uncertainty or randomness in a probability distribution. It quantifies the amount of information required to describe the state of a system. For a discrete random variable $X$ with possible outcomes (states) ${x_1, x_2, \dots, x_n}$ and corresponding probabilities $P(X=x_i) = p_i$, the Shannon entropy $H(X)$ is defined as:

$$H(X) = -\sum_{i=1}^n P(x_i) \log_2 P(x_i)$$

### Kullback-Leibler Divergence

KL divergence is a measure of how one probability distribution diverges from a second, expected probability distribution. It quantifies the amount of information lost when approximating one distribution with another. Mathematically, for two discrete probability distributions $P(X)$ and $Q(X)$ over the variable $X$, the KL divergence from $Q$ to $P$ is defined as (note here we use the log base 2 to represent bits, where the base e for "nats" is sometimes used):

$$D_{KL}(P(X)||Q(X)) = \sum_{i=1}^n P(x_i) \log_2\frac{P(x_i)}{Q(x_i)}$$

In the context of observed streamflow, KL divergence can be used to compare the true distribution of streamflow values at a given location (derived from measurements) to a modelled or assumed distribution. For example, given a simple model that predicts streamflow at ungauged locations, KL divergence helps us quantify how much information is missing or lost when a model’s predictions are used instead of having actual observed values.  It represents the extra number of yes or no questions you would have to ask on average to get the correct answer above the minimum number of yes or no questions required given perfect frequency estimates.  The optimal sequence of yes or no questions simply divides the overall probability into as close to 50/50 as possible each time until an unambiguous answer is arrived at.  Imperfect frequency estimates result in sub-optimally dividing the space when asking yes/no questions, so a few more questions must be asked in the long run.  The children's board game Guess Who is an excellent example.

* In the information theory literature, the KL divergence is described as the inefficiency in the encoding dictionary due to incorrect estimation of the frequency of symbols or states.
    * The Kolmogorov complexity represents a lower bound on optimal encoding.
    * The optimality of encoding translates to the average length of message transmitted, expressed in bits
    * A model that is better than random guessing represents signal compression
    * There is a fundamental link between compression and prediction.  "A model that compresses well generalizes well" (HUTTER).

### Total Variation Distance

Total Variation Distance (TVD) is a measure of the difference between two probability distributions. It quantifies the maximum difference in probabilities assigned to the same events (states) by two distributions. For two discrete probability distributions $P$ and $Q$ over the variable $X$, the TVD is defined as:

$$\text{TVD}(P,Q) = \frac{1}{2}\sum_i|P(x_i) - Q(x_i)|$$

In the context of observed streamflow, TVD can be used to compare the true distribution of streamflow values at a given location (derived from measurements) to a modelled or assumed distribution. For example, if we have a model that predicts streamflow at various locations, TVD describes the **maximum discrepancy** between the model’s predictions and the actual observed values.

### Earth Mover's Distance

The Earth Mover's Distance (EMD), is a (bipartite) network flow minimization problem.  Bipartite simply means we have graphs, in this case probability mass functions of the proxy and target runoff distributions $P$ and $Q$, with edges joining $p_i$ to $q_i$ and no edges within $P$ or within $Q$.  

The distance describes the difference between the two probability distributions in terms of the total distance the probability mass must be moved to transform one distribution to another. For two discrete probability distributions $P$ and $Q$ over the variable $X$, the EMD is defined from {cite}`emd_1998` as the minimum cost:

$$\sum_{i\in P} \sum_{j\in Q} c_{ij}f_{ij}$$

Subject to:

$$f_{ij} \geq 0 \quad i\in P, j \in Q$$
$$\sum_{i\in P} f_{ij} = y_j \quad j \in Q$$
$$\sum_{j\in Q} f_{ij} \leq x_i \quad i\in P$$

Where $x_i$ is the frequency of the proxy $p_i$ and $y_j$ is the "capacity" if each state $q_i$, or the difference between $p_i$ and $q_i$.  The first constraint prevents negative flows $f_{ij}$, which means balances can't be met by moving "mass" from $p_i$ to $q_i$, and instead it must be moved "laterally" at a cost of $c_{ij}$, or the distance from state $i$ to state $j$.

In the context of observed streamflow, the EMD can be used to compare the true distribution of unit area runoff at a given location (derived from measurements) to a modelled or assumed distribution. For example, if we have a model that predicts streamflow at various locations, the EMD helps us quantify how different the predicted distribution is from the distribution of observed values in the same units of the distributions, in this case UAR ($L/s/\text{km}^2$).  

## Predictive Model

### Gradient Boosted Decision Trees

Gradient Boosting Decision Tree (GBDT) is a widely used machine learning algorithm that builds an ensemble of (simple) decision trees in a sequential manner. Each tree is trained to correct the errors of the previous trees, gradually improving the overall prediction accuracy. The method combines the strengths of decision trees with boosting, a technique that focuses on balancing the model's bias and variance.

General procedure:

1. **Initialization**: The algorithm starts with an initial prediction, often the mean of the target values.
2. **Iterative Learning**: In each iteration, a new decision tree is trained to predict the residual errors (differences between the actual values and the current predictions).
3.  **Gradient Descent**: The algorithm uses gradient descent to minimize the loss function, guiding the training of each new tree to correct the errors of the ensemble.
4. **Combination**: The predictions of all trees are combined to form the final prediction, typically by summing the weighted outputs of the trees.

The GBDT model is selected for their strength in detecting nonlinear relationships from high-dimensional input feature sets, for the ability to set up training and testing for robust model training, and for the ability to test relative importance of features.  One advantage of GBDT over the random forest (RF) approach used in {cite}`addor2018ranking` for hydrological signature prediction from attributes is **the training data is not limited by incomplete feature sets** -- rather the ensemble tree construction method, which uses random subsamples of both rows and columns, allows samples with missing attributes to remain in the training data.



### Application of GBDT for Predicting Entropy and Divergence Measures

In the context of predicting various information and divergence measures, GBDT can be used to model complex relationships between features and target variables. We use GBDT models to capture nonlinear patterns and predict KL divergence, Total Variation Distance, and Earth Mover's Distance by learning from a large set of catchment attributes.  

### Model Validation / Testing

To address the problem of overfitting, 5-fold cross validation is done on a training dataset which is separated at the outset from a test set to prevent information leakage from training to validation.  The GBDT model procedure is carried out as follows: 

1. Split the data into 90% training and 10% testing.  Note that the split is done by station id, not by row since rows contain pairwise information.  Since data from two stations is contained in each row, separating by station ID ensures that data from stations in the training set do not "leak" into the held-out test set.  
2. Run a number of cross validation tests on the training data while randomly permuting the gradient boosting hyperparameters.  The purpose of this step is to a) optimize over the hyperparameters, and b) find the hyperparameter set that yields the best average validation score.
3. Retrain a model using the full training set based on the (approximately) optimal hyperparameters, generate predictions on the hold-out test set to determine the model performance on unseen data.
4. (Possibly) repeat steps 1-3 several times to evaluate selection bias in the hold-out set. 

## Graph Partitioning

The first two predictors are computed on data from individual streamflow monitoring locations.  The subsequent three predictors (f-divergences) are computed on pairs of catchments.  This can be represented as a network graph:

### Train-Test Split

The input dataset is pairwise comparisons of just over 1300 (streamflow) monitored catchments, their attributes, and the attribute differences.  After filtering for data concurrency (minimum 1 year, < 5 days missing per month) and maximum distance between basin centroids (500 km) we are left with roughly 225K pairs.  The pairwise setup means that station data appears in more than one row.  As a result, the attributes of stations can end up in both training and test sets if we simply split by randomly assigning rows to training or test sets.   We can't simply cut edges until the graph is separated because it is a [generalization of the "keeping a subset of vertices" problem in graph theory which is NP-hard](https://en.wikipedia.org/wiki/Independent_set_(graph_theory)).

To address this issue we split the dataset spatially to create partitioned datasets for each fold and draw samples from within the "cluster" while filtering all edges between the fold and the rest of the set.  The end goal is to generate training folds where the official_id does not appear in both training and test set, in either proxy or target column.  One problem remains, and that is to generate training and test sets with some assurance that the target variable distributions match to some degree, but this is less critical than ensuring there is no data leakage between training and test data.  

Next we create training/test splits and visualize how the target variable distributions compare.


### Pre-load results

In [120]:
BASE_DIR = os.getcwd()

tiles = xyz['USGS']['USTopo']
result_dict = {}
nrows = None
# open an example pairwise results file
input_folder = os.path.join(
    BASE_DIR, "data", "processed_divergence_inputs",
)

# the input data file has an associated revision date
revision_date = '20240812'

all_test_results = {}
attribute_set_names = ['climate', '+terrain', '+land_cover', '+soil']

In [121]:
for bitrate in range(4,13):
    if bitrate in [5, 7]:
        continue
    print(f'bitrate = {bitrate}')
    fname = f"KL_results_{bitrate}bits_{revision_date}.csv"
    # if partial_counts is true, we load a separate result file
    # where observation counts incorporated a 10% uniform uncertainty
    whole_counts_fname = f"KL_results_{bitrate}bits_{revision_date}.csv"
    partial_counts_fname = f"KL_results_{bitrate}bits_{revision_date}_partial_counts.csv"

    wc_input_data_fpath = os.path.join(input_folder, whole_counts_fname)
    pc_input_data_fpath = os.path.join(input_folder, partial_counts_fname)
    
    df_partial = pd.read_csv(pc_input_data_fpath, nrows=nrows, low_memory=False)
    df_whole = pd.read_csv(wc_input_data_fpath, nrows=nrows, low_memory=False)
    
    result_dict[bitrate] = {'partial': df_partial, 'whole': df_whole}

bitrate = 4
bitrate = 6
bitrate = 8
bitrate = 9
bitrate = 10
bitrate = 11
bitrate = 12


In [122]:
import geopandas as gpd

from shapely.geometry import Point

from bokeh.plotting import figure, show
from bokeh.layouts import gridplot, row, column
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
from bokeh.palettes import Sunset10, Vibrant7, Category20, Bokeh6, Bokeh7, Bokeh8

from sklearn.cluster import AgglomerativeClustering
output_notebook()

Loading BokehJS ...

In [123]:
# load the catchment characteristics
fname = 'BCUB_watershed_attributes_updated.csv'
attr_df = pd.read_csv(os.path.join('data', fname))
attr_df.columns = [c.lower() for c in attr_df.columns]
station_ids = attr_df['official_id'].values
print(f'There are {len(station_ids)} monitored basins in the attribute set.')

There are 1325 monitored basins in the attribute set.


In [124]:
n_folds = 20
linkage = 'ward'
attr_gdf = attr_df[['official_id', 'centroid_lat_deg_n', 'centroid_lon_deg_e']].copy()
lon, lat = 'centroid_lon_deg_e', 'centroid_lat_deg_n'
agg_clustering = AgglomerativeClustering(linkage=linkage, distance_threshold=None, n_clusters=n_folds)
attr_gdf['cluster'] = agg_clustering.fit_predict(attr_gdf[[lon, lat]])
attr_gdf['geometry'] = attr_gdf.apply(lambda row: Point(row[lon], row[lat]), axis=1)
attr_gdf = gpd.GeoDataFrame(attr_gdf, crs='EPSG:4326')
attr_gdf.to_crs(3857, inplace=True)

min_points_per_cluster = 30

# Post-processing: Check and handle clusters with fewer points than the threshold
cluster_sizes = attr_gdf['cluster'].value_counts()

# Find clusters with fewer points than the minimum threshold
small_clusters = cluster_sizes[cluster_sizes < min_points_per_cluster].index

# Find points in the larger clusters
large_clusters = cluster_sizes[cluster_sizes >= min_points_per_cluster].index
large_cluster_points = attr_gdf[attr_gdf['cluster'].isin(large_clusters)]

# Reassign points in small clusters individually
for small_cluster in small_clusters:
    # Find points in the small cluster
    small_cluster_points = attr_gdf[attr_gdf['cluster'] == small_cluster]
    
    # For each point in the small cluster, find the nearest point in the large clusters
    for idx, row in small_cluster_points.iterrows():
        # Compute distances to all points in the larger clusters
        distances = large_cluster_points.geometry.distance(row['geometry'])
        
        # Find the nearest large cluster
        nearest_large_cluster = large_cluster_points.loc[distances.idxmin(), 'cluster']
        
        # Reassign this point to the nearest large cluster
        attr_gdf.at[idx, 'cluster'] = nearest_large_cluster

# Output the corrected clusters and remap to make continuous numbering
# unique_clusters = attr_gdf['cluster'].value_counts()
unique_clusters = sorted(attr_gdf['cluster'].unique())
cluster_mapping = {old: new for new, old in enumerate(unique_clusters)}  # Create mapping old -> new
attr_gdf['cluster'] = attr_gdf['cluster'].map(cluster_mapping)  # Apply the renumbering
attr_gdf.head()

,official_id,centroid_lat_deg_n,centroid_lon_deg_e,cluster,geometry
0,05010500,48.967598,-113.793846,11,POINT (-12667472.958 6269365.277)
1,05012000,48.979019,-113.856276,11,POINT (-12674422.679 6271302.191)
2,05014000,48.763896,-113.718453,11,POINT (-12659080.244 6234894.012)
3,05014500,48.782935,-113.706072,11,POINT (-12657702.078 6238109.974)
4,05017500,48.721647,-113.566857,11,POINT (-12642204.692 6227762.064)


In [125]:
def format_fig_fonts(fig, font_size=20, font='Bitstream Charter', legend=True):
    fig.xaxis.axis_label_text_font_size = f'{font_size}pt'
    fig.yaxis.axis_label_text_font_size = f'{font_size}pt'
    fig.xaxis.major_label_text_font_size = f'{font_size-2}pt'
    fig.yaxis.major_label_text_font_size = f'{font_size-2}pt'
    fig.yaxis.axis_label_text_font = font
    fig.xaxis.axis_label_text_font = font
    fig.xaxis.major_label_text_font = font
    fig.yaxis.major_label_text_font = font
    if legend == True:
        fig.legend.label_text_font_size = f'{font_size-2}pt'
        fig.legend.label_text_font = font
    return fig
    

In [126]:
f_unique, f_counts = np.unique(attr_gdf['cluster'], return_counts=True)
count_sum = sum(f_counts)
n_clusters_created = len(f_counts)
print(f'{n_clusters_created} clusters, {count_sum} total stations')
f_counts

15 clusters, 1325 total stations


array([131,  83, 106, 251,  96,  97,  75,  64,  38,  41,  81,  70,  33,
       103,  56])

### Plot the spatial partitions

In [127]:
# Assuming 'attr_gdf' is your GeoDataFrame and 'cluster' is the column for cluster IDs
# Convert to a ColumnDataSource for Bokeh
attr_gdf['lat'] = attr_gdf.geometry.y
attr_gdf['lon'] = attr_gdf.geometry.x
foo = attr_gdf[[c for c in attr_gdf.columns if c != 'geometry']].copy()
source = ColumnDataSource(foo)

# Define glyph shapes and colors
glyphs = ['circle', 'square', 'triangle', 'diamond', 'inverted_triangle']  # glyphs
colors = Category20[15]  # 2 colors * 5 glyphs = 10 unique symbols

# Create a function to map clusters to glyph and color
def map_cluster_to_glyph_color(cluster_id):
    cluster_id = int(cluster_id)  # Ensure cluster_id is an integer
    return glyphs[cluster_id % len(glyphs)], colors[cluster_id % len(colors)]

# Create a Bokeh plot
p = figure(title="Spatial Partitioning of Separate Clusters", 
           tools="pan,wheel_zoom,reset", match_aspect=True, 
           width=900, height=650)

# Create a ColumnDataSource for plotting
for cluster_id in attr_gdf['cluster'].unique():
    # Get marker and color for each cluster
    marker, color = map_cluster_to_glyph_color(cluster_id)
    
    # Filter data for the current cluster
    cluster_data = attr_gdf[attr_gdf['cluster'] == cluster_id][[c for c in attr_gdf.columns if c != 'geometry']]
    
    # Plot using scatter with the marker and color
    p.scatter('lon', 'lat', source=cluster_data, marker=marker, size=5, 
              color=color, legend_label=f'{cluster_id}',
             line_color='black', line_alpha=0.5, line_width=1)

# Customize the plot
p.add_tile(tiles, retina=True)
p.grid.visible = False
# Customize and sort the legend
p.legend.title = "Fold #"
p.legend.ncols = 1
p.legend.label_text_font_size = '8pt'
p.legend[0].items = sorted(p.legend[0].items, key=lambda t: f'{int(t.label.value):02d}')
p.add_layout(p.legend[0], 'right')

# Show the plot
show(p)

### In the network graph, find the "exceedance" of connection distances

In [28]:
graph_df = result_dict[bitrate]['whole'].copy()
# sort the values
dist_based_samples = []
distance_limits = np.arange(50, 1010, 10)
for max_distance in distance_limits:
    n_edges = len(graph_df[graph_df['centroid_distance'] <= max_distance])
    dist_based_samples.append(n_edges)


In [29]:
dfig = figure(title='', width=800, height=600)
src = ColumnDataSource({'x': distance_limits, 'y': dist_based_samples})
dfig.line('x', 'y', color='dodgerblue', line_width=3, source=src)
dfig.xaxis.axis_label = r'$$\text{Max. Distance [km]}$$'
dfig.yaxis.axis_label = r'$$\text{Sample Size}$$'
dfig = format_fig_fonts(dfig, legend=False)

show(dfig)

In [30]:
# edge_counts = pd.concat([graph_df['proxy'], graph_df['target']]).value_counts()
# Function to compute the CDF
def compute_cdf(data):
    sorted_data = np.sort(data)
    cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
    return sorted_data, cdf

# Initialize a dictionary to hold CDF data for each threshold
cdf_data = {}
# Define distance thresholds for the CDFs
distance_thresholds = np.arange(100, 1100, 100)
# Loop through each distance threshold and compute the CDF of the number of edges per node
for max_distance in distance_thresholds:
    # Filter the DataFrame for the current threshold
    filtered_df = graph_df[graph_df['centroid_distance'] <= max_distance]
    
    # Count the number of edges per node (degree)
    edge_counts = pd.concat([filtered_df['proxy'], filtered_df['target']]).value_counts()
    
    # Compute the CDF for this threshold
    sorted_counts, cdf = compute_cdf(edge_counts)
    
    # Store the data for plotting later
    cdf_data[max_distance] = (sorted_counts, cdf)

# Plot the CDF for each distance threshold
cplt = figure(title='CDF of Edges per Node by Distance Threshold', 
              # x_axis_type='log', 
              width=900, height=600)

n=0
for max_distance in distance_thresholds:
    sorted_counts, cdf = cdf_data[max_distance]
    cplt.line(sorted_counts, cdf, line_width=3,
              legend_label=f'd <= {max_distance}km', color=Sunset10[n])
    n += 1

# Add plot labels and legend
cplt.xaxis.axis_label = r'$$\text{Number of Edges (Degree)}$$'
cplt.yaxis.axis_label = r'$$P(X \leq x)$$'
cplt.add_layout(cplt.legend[0], 'right')
cplt = format_fig_fonts(cplt)
show(cplt)

## Visualize the Spatial Distribution of Degree

The degree represents the number of edges connecting a node to the rest of the graph.   Determine the number of edges from each node and plot a heat map to reflect the spatial distribution of degree.  Set a logartithmic colour bar to highlight the wide range of network connectivity.  Set a few distance limits to see how the network connectivity changes in space when varying the distance threshold.

In [128]:
# Compute degree
def compute_degree(graph_df, distance_threshold=1000):
    df = graph_df[graph_df['centroid_distance'] <= distance_threshold].copy()
    in_degree = df['target'].value_counts()
    out_degree =df['proxy'].value_counts()
    degree = in_degree.add(out_degree, fill_value=0).reset_index()
    degree.columns = ['node', 'degree']
    degree['lon'] = degree.apply(lambda r: attr_gdf.loc[attr_gdf['official_id'] == r['node'], 'geometry'].x.values[0], axis=1)
    degree['lat'] = degree.apply(lambda r: attr_gdf.loc[attr_gdf['official_id'] == r['node'], 'geometry'].y.values[0], axis=1)
    return degree

In [129]:
from bokeh.models import (LogColorMapper, ColorBar, LogColorMapper, Whisker,
                        HoverTool, NumeralTickFormatter, PrintfTickFormatter)
from bokeh.palettes import Sunset10
from bokeh.transform import log_cmap, linear_cmap

def ticker(tick):    
    return f'{tick:.1e}'

degree_figs = []
n = 0
for dist_threshold in [1000, 700, 400, 100]:
    degree = compute_degree(graph_df, dist_threshold)
    n_nodes = len(degree)
    p = figure(width=800, height=600, title=f"Centroid dist <= {dist_threshold}km (N={n_nodes})", 
               x_axis_label='Longitude', y_axis_label='Latitude')
    p.add_tile(tiles, retina=True, alpha=0.5)
    
    # Prepare the data source for Bokeh
    source = ColumnDataSource(degree)
    color_mapper = linear_cmap(field_name='degree', palette=Sunset10, 
                               low=degree['degree'].min(), high=degree['degree'].max())
    
    # Scatter plot with discrete colors
    p.scatter('lon', 'lat', size=4., source=source,
              fill_color=color_mapper,
              line_color='black', line_alpha=0.35, alpha=0.6)
    
    # # Add a color bar
    color_bar = ColorBar(color_mapper=color_mapper['transform'], label_standoff=10, width=8, location=(0, 0),
                        major_label_text_font='Bitstream Charter', major_label_text_font_size='16pt', title='Node Degree',
                        title_text_font='Bitstream Charter', title_text_font_size='16pt')
    p.add_layout(color_bar, 'right')
    p.xaxis.formatter = PrintfTickFormatter(format='%.1e')
    p.yaxis.formatter = PrintfTickFormatter(format='%.1e')
    # Add a hover tool to display node and degree
    hover = HoverTool()
    hover.tooltips = [
        ("Node", "@node"),
        ("Degree", "@degree")
    ]
    if n in [0, 1]:
        p.xaxis.visible = False
    if n in [1, 3]:
        p.yaxis.visible = False
        
    n += 1
    p.add_tools(hover)
    degree_figs.append(p)
    

In [33]:
layout = gridplot(degree_figs, ncols=2, width=450, height=300)
show(layout)

Above, we see that restricting the maximum distance significantly reduces the degree of nodes.  If we restrict it to far, i.e. $(d < ~200) $ km, we begin to isolate nodes (stations) completely.

In [130]:
def plot_kld_by_distance(result_dict, n_bins=25, bitrate=4, prior=0, counts='whole', concurrent='concurrent', show_legend=True):
    """
    Creates a plot showing the 95% confidence interval for KL-divergence
    (for some bitrate and prior) across discretized bins of 'centroid_distance'.
    
    Parameters:
        results_dict (dict): The results dictionary with 'centroid_distance' 
                            and 'kl_divergence_<>_<>' columns.
        n_bins (int): The number of bins to discretize 'centroid_distance' into.
    
    Returns:
        plot: Bokeh plot object.
    """
    p = figure(width=600, height=350, y_range=(0.0001, 20), y_axis_type='log',
               x_axis_label='Centroid Distance (Binned)', y_axis_label='KL Divergence [bits/sample]')
    ci_plotted = False
    for c in ['whole', 'partial']:
        for d in ['concurrent', 'nonconcurrent']:
            line_dash = 'dashed'
            if c == 'whole':
                line_dash = 'solid'
            clr = 'orange'
            if d == 'concurrent':
                clr = 'dodgerblue'

            label = f'{d}_{c}counts'
    
            data = result_dict[bitrate][c].copy()
            dkl_col = f'dkl_{d}_post_{prior}R'
            data.dropna(subset=[dkl_col], inplace=True)
            n_sample = int(len(data) / n_bins)

            p.title=f"{bitrate}bit, 10^{prior}pr, (N={n_sample}/bin)"
    
            assert dkl_col in data.columns
            
            # Discretize 'centroid_distance' into n_bins of equal sample size
            data['distance_bin'] = pd.qcut(data['centroid_distance'], q=n_bins, labels=False)
            
            # Calculate the mean and 95% confidence interval using percentiles for each bin
            summary = data.groupby('distance_bin').apply(
                lambda x: pd.Series({
                    'mean': x[dkl_col].mean(),
                    'median': np.percentile(x[dkl_col], 50),
                    'ci_lower': np.percentile(x[dkl_col], 2.5),
                    'ci_upper': np.percentile(x[dkl_col], 97.5),
                    'centroid_distance': x['centroid_distance'].mean()
                })
            ).reset_index()
            # Prepare the data source for Bokeh
            source = ColumnDataSource(summary)
        
            min_d, max_d = summary['ci_lower'].min() * 0.95, summary['ci_upper'].max() * 1.1
            
        
            # Add whiskers for the confidence intervals
            if not ci_plotted:
                p.varea(x='centroid_distance', y1='ci_lower', y2='ci_upper', source=source, alpha=0.5,
                        legend_label='95% CI', color='grey')
                ci_plotted = True
            # whisker = Whisker(base='centroid_distance', upper='ci_upper', lower='ci_lower', 
            #                   legend_label='95% CI', source=source)
            # whisker.line_width = 1
            # p.add_layout(whisker)
    
            # Add the mean values as points
            # p.line('centroid_distance', 'mean', 
            #           line_width=3, source=source, color='dodgerblue', legend_label='Mean')
            p.line('centroid_distance', 'median', 
                      line_width=3, source=source, color=clr, legend_label=label, line_dash=line_dash)
    if show_legend is True:
        p.legend.location = 'bottom_right'
    else:
        p.legend.visible = False
    p.legend.background_fill_alpha = 0.4
    return p


In [131]:
plts = []
show_legend = True
for b in [4, 6, 8, 10, 12]:
    for pr in [-2, 0, 2]:
        # hide_y = False
        counts, d = 'whole', 'concurrent'
        p1 = plot_kld_by_distance(result_dict, bitrate=b, counts=counts, prior=pr, concurrent=d, show_legend=show_legend)
        plts.append(p1)
        show_legend=False
    # hide_y = True

In [132]:
layout = gridplot(plts, ncols=3, width=400, height=300)
show(layout)

## Graph Partitioning Methods

In order to propertly test out of sample, we must ensure that data from the training set are not included in the test set.  In the case of the f-divergence measures, each sample contains information about two streamflow monitoring locations, the target and the proxy.  This problem represents a network graph, making the training/test separation a graph partitioning problem.  

In [133]:
prior = 0
counts = 'whole'
concurrent = 'concurrent'
bitrate = 8

# for the clusters created above, iterate through and compute 
# the cdf for DKL in-group vs. out-group samples
cluster_cdfs = []
all_cluster_ids = []
dkl_col = f'dkl_{concurrent}_post_{prior}R'
# Create a Bokeh plot
p = figure(width=800, height=500, title=f"Empirical CDF of {dkl_col} (AND vs. NOR)",
           x_axis_label=dkl_col, y_axis_label='Empirical CDF')

for cluster_id in sorted(attr_gdf['cluster'].unique()):
    # Get marker and color for each cluster
    marker, color = map_cluster_to_glyph_color(cluster_id)
    
    # Filter data for the current cluster
    cluster_data = attr_gdf[attr_gdf['cluster'] == cluster_id][[c for c in attr_gdf.columns if c != 'geometry']].copy()
    cluster_ids = cluster_data['official_id']
    all_cluster_ids += list(cluster_ids)
    
    # print(cluster_data.head(3))
    dkl_sample = result_dict[bitrate][counts].copy()
    dkl_sample.dropna(subset=[dkl_col], inplace=True)
    dkl_sample_AND = dkl_sample[(dkl_sample['proxy'].isin(cluster_ids)) & (dkl_sample['target'].isin(cluster_ids))]
    dkl_sample_NOR = dkl_sample[(~dkl_sample['proxy'].isin(cluster_ids)) & (~dkl_sample['target'].isin(cluster_ids))]

    percentiles = np.linspace(0, 100, 200)

    # Compute the percentile values for each dataset
    percentile_values_and = np.percentile(dkl_sample_AND[dkl_col], percentiles)
    percentile_values_nor = np.percentile(dkl_sample_NOR[dkl_col], percentiles)

    # Create CDF values for each set of percentiles (from 0 to 1)
    cdf_values = percentiles / 100

    # Prepare data for Bokeh
    source_and = ColumnDataSource(data={'x': percentile_values_and, 'y': cdf_values})
    source_nor = ColumnDataSource(data={'x': percentile_values_nor, 'y': cdf_values})
    
    # Plot the CDFs
    p.line('x', 'y', source=source_and, line_width=2, color='blue', legend_label=f'AND', alpha=0.5)
    p.line('x', 'y', source=source_nor, line_width=2, color='green', legend_label=f'NOR', alpha=0.5)
    
    # Show the plot
    p.legend.location = "bottom_right"

p.xaxis.axis_label = 'KL Divergence [bits/sample]'
p.yaxis.axis_label = 'P(X>=x)'
p = format_fig_fonts(p)

In [40]:
show(p)

### Plot the CDFs of KL divergence by bitrate and prior


In [51]:
def compute_empirical_cdf(data):
    percentiles = np.linspace(0, 100, 200)
    # sorted_data = np.sort(data)
    sorted_data = np.percentile(data, percentiles)
    # cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
    cdf = percentiles / 100.0
    return sorted_data, cdf

In [71]:
dkl_cdfs = []
for bitrate in [4, 6, 8, 10, 12]:
    fig = figure(title=f'DKL CDF: {bitrate}bits', x_range=(0, 20))
    n = 0
    for prior in [-2, -1, 0, 1, 2]:            
        target_col = f'dkl_concurrent_post_{prior}R'
        data = result_dict[bitrate]['whole'][[target_col]].copy()
        data.dropna(inplace=True)
        xx, yy = compute_empirical_cdf(data.values)
        fig.line(xx, yy, color=Sunset10[n], legend_label=f'10^{prior}', line_width=3)
        n += 2
    dkl_cdfs.append(fig)

In [82]:
layout = gridplot(dkl_cdfs, ncols=3, width=400, height=275)
show(layout)

In [73]:
dkl_cdfs = []

for prior in [-2, -1, 0, 1, 2]: 
    fig = figure(title=f'DKL CDF: 10^{prior} prior')
    n = 0 
    for bitrate in [4, 6, 8, 10, 12]:        
        target_col = f'dkl_concurrent_post_{prior}R'
        data = result_dict[bitrate]['whole'][[target_col]].copy()
        data.dropna(inplace=True)
        xx, yy = compute_empirical_cdf(data.values)
        fig.line(xx, yy, color=Sunset10[n], legend_label=f'{bitrate} bits', line_width=3)
        n += 2
    dkl_cdfs.append(fig)

In [80]:
layout = gridplot(dkl_cdfs, ncols=3, width=400, height=275)
show(layout)

From the above plot, we can see that the similarity of distributions between partitions is affected by the clustering / number of folds.

In [386]:
# save these cross-validation folds for the f-divergence train/test split
attr_gdf.to_file(f'data/stn_attributes_with_assigned_cluster_{n_clusters_created}.geojson')

### Map Colouring Partition

Instead of spatial cluster-based partitioning, try testing a partitioning using N classes partitioned by alternating classes spatially.  This will result in classes that cover the full study region but with half the points.

In [75]:
def alternate_partition_geopandas(gdf, n_classes=2):
    # Extract coordinates from the GeoDataFrame
    gdf['coords'] = gdf.geometry.apply(lambda geom: (geom.x, geom.y))
    
    # Sort by coordinates, for example, first by x and then by y
    sorted_gdf = gdf.sort_values(by=['coords'])
    
    # Alternate assignment of nodes to N classes
    sorted_gdf['class'] = np.arange(len(sorted_gdf)) % n_classes
    
    return sorted_gdf

In [77]:
n_classes = 5
sorted_gdf = attr_gdf.sort_values(by=['lon', 'lat'])
# Alternate assignment of nodes to N classes
sorted_gdf[f'{n_classes}_spatial'] = np.arange(len(sorted_gdf)) % n_classes

In [84]:
# Create a Bokeh plot
p = figure(title="Spatial Partitioning II", 
           tools="pan,wheel_zoom,reset", match_aspect=True, 
           width=900, height=650)

# Create a ColumnDataSource for plotting
for cluster_id in sorted_gdf[f'{n_classes}_spatial'].unique():
    # Get marker and color for each cluster
    marker, color = map_cluster_to_glyph_color(cluster_id)
    
    # Filter data for the current cluster
    cluster_data = sorted_gdf[sorted_gdf[f'{n_classes}_spatial'] == cluster_id][[c for c in attr_gdf.columns if c != 'geometry']]
    
    # Plot using scatter with the marker and color
    p.scatter('lon', 'lat', source=cluster_data, marker=marker, size=5, 
              color=color, legend_label=f'{cluster_id}',
             line_color='black', line_alpha=0.5, line_width=1)

# Customize the plot
p.add_tile(tiles, retina=True)
p.grid.visible = False
# Customize and sort the legend
p.legend.title = "Fold #"
p.legend.ncols = 1
p.legend.label_text_font_size = '8pt'
p.legend[0].items = sorted(p.legend[0].items, key=lambda t: f'{int(t.label.value):02d}')
p.add_layout(p.legend[0], 'right')

# Show the plot
show(p)

In [134]:
prior = 0
counts = 'whole'
concurrent = 'concurrent'
bitrate = 8

# for the clusters created above, iterate through and compute 
# the cdf for DKL in-group vs. out-group samples
cluster_cdfs = []
all_cluster_ids = []
dkl_col = f'dkl_{concurrent}_post_{prior}R'
# Create a Bokeh plot
p = figure(width=800, height=500,# title=f"Empirical CDF of {dkl_col} (AND vs. NOR)",
           x_axis_label=dkl_col, y_axis_label='Empirical CDF')

for cluster_id in sorted_gdf[f'{n_classes}_spatial'].unique():
    # Get marker and color for each cluster
    marker, color = map_cluster_to_glyph_color(cluster_id)
    
    # Filter data for the current cluster
    cluster_data = sorted_gdf[sorted_gdf[f'{n_classes}_spatial'] == cluster_id][[c for c in sorted_gdf.columns if c != 'geometry']].copy()
    cluster_ids = cluster_data['official_id']
    all_cluster_ids += list(cluster_ids)
    
    # print(cluster_data.head(3))
    dkl_sample = result_dict[bitrate][counts].copy()
    dkl_sample.dropna(subset=[dkl_col], inplace=True)
    # test set
    dkl_sample_AND = dkl_sample[(dkl_sample['proxy'].isin(cluster_ids)) & (dkl_sample['target'].isin(cluster_ids))]
    # training set
    dkl_sample_NOR = dkl_sample[(~dkl_sample['proxy'].isin(cluster_ids)) & (~dkl_sample['target'].isin(cluster_ids))]
    print(f'{len(dkl_sample_NOR)}/{len(dkl_sample_AND)} train/test sample sizes')

    percentiles = np.linspace(0, 100, 200)

    # Compute the percentile values for each dataset
    percentile_values_and = np.percentile(dkl_sample_AND[dkl_col], percentiles)
    percentile_values_nor = np.percentile(dkl_sample_NOR[dkl_col], percentiles)

    # Create CDF values for each set of percentiles (from 0 to 1)
    cdf_values = percentiles / 100

    # Prepare data for Bokeh
    source_and = ColumnDataSource(data={'x': percentile_values_and, 'y': cdf_values})
    source_nor = ColumnDataSource(data={'x': percentile_values_nor, 'y': cdf_values})
    
    # Plot the CDFs
    p.line('x', 'y', source=source_and, line_width=2, color='dodgerblue', legend_label=f'Test', alpha=0.75)
    p.line('x', 'y', source=source_nor, line_width=2, color='magenta', legend_label=f'Train', alpha=0.85)
    
    
    
    # Show the plot
    p.legend.location = "bottom_right"

p.xaxis.axis_label = 'KL Divergence [bits/sample]'
p.yaxis.axis_label = r'$$P(X \geq x)$$'
p.legend.click_policy = 'hide'
p = format_fig_fonts(p)

278742/16824 train/test sample sizes
277277/17120 train/test sample sizes
273276/18077 train/test sample sizes
271442/18603 train/test sample sizes
283141/15556 train/test sample sizes


In [135]:
show(p)

In [98]:
sorted_gdf.to_file(f'data/stn_attributes_with_{n_classes}_spatial_partitions.geojson')

In [99]:
unique, counts = np.unique(sorted_gdf[f'{n_classes}_spatial'], return_counts=True)
print(unique)
counts

[0 1 2 3 4]


array([265, 265, 265, 265, 265])

## (Work in Progress)

## Notes on f-divergence and consistency 

f-divergence is any convex function of the likelihood ratio.

### Types of Expectation of the Loss Function

In general, consistency means as the number of data points gets large, your estimate converges to the right answer.  One way to test consistency is to look at bias and variance separately.  $\mathbb{E}_\theta$ is interested in "other sample points beyond the ones you have", while Bayesian focuses on "fixed X", or the sample points one does have.

1.  The unconditional (frequentist) and conditional (Bayesian) expectation of the loss function:

* **Frequentist**: $\mathbb{E }_{\theta} \mathcal{l} \left(\delta(X), \theta \right)$ "expectation over the sample space", theta held fixed, expectation with respect to the $X$, or probability distribution of X "indexed by" theta.
* **Bayesian**: $\mathbb{E} \left[ \mathcal{l} \left(\delta(X), \theta \right) | X \right]$ theta is the random variable, expectation is conditional on X

Where:  
* $\mathcal{l}$ is some loss function (square, absolute, zero-one, etc.),
* $\delta(X)$ is the statistical procedure which is for simplicity also denoted $\hat \theta$,
* $\theta$ is the parameterization of some distribution, and $X$ is the target variable.

E.g. the frequentist risk of the square loss $\mathcal{l}$:
* square loss: $\mathcal{l}(\delta(X)) = \mathcal{l}(\hat \theta, \theta) = (\hat \theta - \theta)^2$
    * the $\hat \theta$ is a random quantity from frequentist point of view, and
    * $\theta$ is a random quantity from the Bayesian point of view.
* The risk: $R_{\theta} = \mathbb{E}_{\theta} \left( \hat \theta - \theta \right)^2 = \mathbb{E}_{\theta} \left( \hat \theta - \mathbb{E}_{\theta} \hat \theta - (\theta - \mathbb{E}_{\theta} \hat \theta) \right)^2 = \mathbb{E}_{\theta} \left( \hat \theta - \mathbb{E}_{\theta} \hat \theta \right)^2 - \left(\theta - \mathbb{E}_{\theta} \hat \theta \right)^2$
    * the first quantity $\mathbb{E}_{\theta} \left( \hat \theta - \mathbb{E}_{\theta} \hat \theta \right)^2$ is the variance
    * the second quantity $\left( \theta - \mathbb{E}_{\theta} \hat \theta \right)^2$ is the squared bias,
    * this is the frequentist expectation because it is over the sample space



### Surrogate Loss Functions, f-Divergences, and Experimental Design


* **Problem**: Find the decision $(Q: \gamma)$ that minimizes the probability of error $P\left(Y \neq \gamma(Z)\right)$

* The decision has two parts: it's determining the quantizer and the discriminant function **jointly**.  Historically one half or other was generally approached.

In statistical machine learning:
* Q is assumed known and the problem is to find the discriminant function $\gamma$
* done by minimization of a "surrogate loss function" such as boosting, logistic regression, SVM.
* decision-theoretic, "consistency results"?

In signal processing:
* all the probabilities are known
* discriminant can be determined by Bayes rule (how?)
* but focuses on getting Q.  "once you get Q, use Bayes rule to get quantizer".
* focused on f-divergenes as a heuristic -- "want to push distributions apart"(?). Then a principle for determining the quantizer is maximizing the f-divergence (or experimental design in general)

Goal is to find the discriminant function and the quantizer that minimize the probability of error.  

Either you assume Q is known and you try to find the discriminant $\gamma$ (machine learning), or "all probabilities are known but focus on getting Q to then arrive at a quantizer".


### Blackwell (1951)

* If a procedure A has a smaller f-divergence than a procedure B (for some fixed $\mathcal{f}$), then there exists some set of prior probabilities such that procedure A has a smaller probability of error than procedure B.
* Given that it is intractable to minimize probability of error, f-divergences have been used as surrogates for probability of error
    * i.e. choose a quantizer Q by maximizing an f-divergence between $P(Z|Y=1)$ and $P(Z|Y=-1)$
        * Hellinger distance (Kalath 1967; Longo et al. 1990)
        * Chernoff distance (Chamberland & Veeravalli, 2003)
 
* Supporting arguments from asymptotics
    * Kullback-Leibler divergence in the Neyman-Pearson setting
    * Chernoff distance in the Bayesian setting
      

From {cite}`nguyen2009surrogate`:

>"An interesting example of this more elaborate formulation is a “distributed detection” problem, in which individual components of the d-dimensional covariate vector are measured at spatially separated locations, and there are communication constraints that limit the rate at which the measurements can be forwarded to a central location where the classification decision is made."

In the example above the d-dimensional covariate vector X represents d independent observation points with their own signals.  The quantizer Q is the decision of how to map the signals of Q to some lower dimension such that a classification 

Given a number of sensors outputting independent signals X, where there is a communication constraint preventing the processing of the complete set of raw signals, a quantizer Q is some (fixed) mapping of X to (lower dimensional) Z.  One half of the problem is to determine Q that minimizes prediction error $P\left(Y \neq \gamma(Z)\right)$ and the other half of the problem is to determine the discriminant function that ...

Can we map the streamflow monitoring network problem in similar terms?

## The General Problem Description

Consider the set of points defining river network confluences as a space of potential locations to monitor streamflow.  A simple approach to predicting streamflow in unmonitored basins (PUB) might be to assume: 
1) a range of streamflow that could occur $[a, b]$, and
2) a prior distribution over some set of states $\omega$ defining streamflow at a point in time.

The states could be defined in relation to some application of interest, or in relation to measurement error/uncertainty.  For example, if the rating curve (stage-discharge relationship) uncertainty is 10% for flows between 0 and 1 $m^3/s$, the appropriate precision is in the order of $0.1 m^3/s$, or to define ten states (discrete intervals) between 0 and 1.  The definition of states is referred to as the quantization of the streamflow signal.

Observing streamflow at one location provides information that can be used to predict streamflow at other locations.  A simple and common first approximation for prediction in ungauged basins (PUB) is to assume the discharge at distinct locations is equal on a unit area basis (UAR model).  This is a reasonable and useful assumption where the locations receive similar climate inputs due to close spatial proximity, or where the catchments are similar in terms of the characteristics related to the mechanisms governing the rainfall runoff response, such as the vegetation, soil, or slope which play a role in how water moves across the surface, into and through the soil, and into the atmosphere.  

A **proxy** location is a monitored catchment whose observations can be used to estimate runoff at a **target** location defining a different catchment. Both locations are watersheds, defined as land areas where all precipitation drains to a common outlet.  A simple PUB model uses the distribution of UAR $(\mathbf{Q})$ at a proxy location to estimate the distribution of UAR $(\mathbf{P})$ at a target location.  Both locations can be characterized by a set of measurable attributes associated with the rainfall-runoff response.

The goal in this exercise is to select the sensor location (or N locations as resources permit) that minimizes the probability of prediction error, in this case incorrect frequency estimation across the unmonitored space.  Stated in another way, the goal is to determine how to prioritize adding stations in such a way as to maximize the expected reduction in uncertainty over the unmonitored space.  The expectation of uncertainty reduction from adding a sensor (monitoring station) is estimated by developing a relationship between catchment attributes and the similarity of $(\mathbf{P})$ and $(\mathbf{Q})$.  In cases where minimizing the probability of error is intractable (support why, namely), f-divergences have been used as surrogates for probability of error (Blackwell 1951, Nguyen/Jordan 2009).  (An f-divergence is defined as any convex function of the likelihood ratio.)

Extending the example of the UAR PUB model above, assuming the maximum uncertainty prior uniform distribution $\mathbb{U}$ over $\omega$ is in many cases a better model for $\mathbf{P}$ than $\mathbf{Q}$.  "Better" in this case means the distributions are more similar, or that the expected message length to transmit observations matching the true distribution is greater than the naive assumption that all states are equally probable.  This feature leads to a possible binary classification /  prediction problem:


## Binary prediction formulation 

Consider an unmonitored stream network where we assume a uniform prior distribution at all possible observation locations.  It is not the case that adding a sensor in some location will provide predictive information for all unmonitored locations.  For many catchments, the uniform prior will be closer to the "true" distribution $\mathbf{P}$.  The term "true" refers to the distribution based on the set of observations held out of sample for computing the loss.  The probability that a potential monitoring location is a "better" proxy than the uniform prior for the UAR distribution at another location describes a binary prediction problem formulation (test the balance of data across bitrates!).  

The discriminant function is what quantifies a proxy distribution as "better", and since the minimization of prediction error in such a formulation is not readily computable or intractable (because?), determine the f-divergence that maps to an equivalent (loss or discriminant) function that is a (consistent) surrogate of prediction error.  The f-divergences are profiled for consistency are the total variation distance (TVD), the Kullback-Leibler divergence (KLD), and the Earth Mover's Distance (EMD), also known as the Wasserstein distance.

Given a covariate vector $\mathbf{X}$ of catchment attributes representing proxy (model) and target (predicted) catchments and a quantization $\mathbf{C}$ mapping streamflow $\mathbf{S}$ from continuous to discrete $\mathbf{Z}$.  $\mathbf{P}$ and $\mathbf{Q}$ are proxy and target distributions of $\mathbf{Z}$.  The discriminant is then a multi-part problem where (what is known and what isn't?) 

From {cite}`nguyen`, the total variational distance is the f-divergence consistent with the 0-1 loss in the in procedures that optimize simultaneously over the discriminant function $\gamma$ and the quantizer $Q$.



* the loss function penalizes incorrect predictions that a proxy location UAR distribution is a better "model" than the uniform distribution when it actually isn't. The goal is to miminize the (Bayes) error $R_{\textbf{Bayes}}(\gamma, C) := \mathbb{P} \left(Y \neq \text{sign}(D_{KL}(\mathbf{P}||\mathbb{U}) - D_{KL}(\mathbf{P}||\mathbf{Q}) \right)$
* For each f-divergence, there is a 1 to many mapping of divergence to loss function that is consistent.  There is a set of loss functions that yield (Bayes) consistency, that is that approach the "true" value as the sample size increases.

$$\text{min } P \left(Y \neq \gamma (X, C(S) \rightarrow Z) \right)$$




$$\text{min } P \left(D_{KL}(P||Q) < D_{KL}(P|| \mathbb{U} ) |X \right)$$

### Universal equivalence among surrogate loss functions

For a given (general, i.e. not necessarily 0-1) surrogate loss function, $\phi$-risk cak be represented as $R_{\phi}(\gamma, \mathbf{Q}) = \sum_x \phi (\gamma(z))\mu(z) + \phi(-\gamma(z))\pi(z)$

Take the infimum over the discriminant function:
* **Infimum (inf)**: The infimum of a set is the greatest value that is less than or equal to every element of the set. It is a type of "lower bound." In cases where the set has a minimum value, the infimum is equal to the minimum. However, if the set does not have a minimum (but has a lower bound), the infimum is the largest value that is not exceeded by any element of the set.

## Citations

```{bibliography}
:filter: docname in docnames
```